# Imports:

In [13]:
import pandas as pd
import numpy as np

# Read the input traning and test data

In [14]:
test_df = pd.read_csv("./car/test.csv", header=None)
train_df = pd.read_csv("./car/train.csv", header=None)

In [119]:
print("train summary:")
print(train_df.describe())
print("===============")
print("test summary:")
print(test_df.describe())

train summary:
            0     1     2     3     4     5      6
count    1000  1000  1000  1000  1000  1000   1000
unique      4     4     4     3     3     3      4
top     vhigh  high     4     4   big   med  unacc
freq      259   255   253   337   341   344    698
test summary:
          0      1      2     3    4     5      6
count   728    728    728   728  728   728    728
unique    4      4      4     3    3     3      4
top     low  vhigh  5more  more  med  high  unacc
freq    190    188    190   248  247   249    512


# Helper functions:

In [120]:
# calculates the majority error
def majority_error():
    print("majority_error")

# calculates the information gain
def entropy(S, Attribute_col_index, Label_col_index):
    print("entropy")
    
    
    
# calculates the gini index
def gini():
    print("gini")

# returns the column index of the best splitter attribute
def Best_spliter_attribute(S, Attributes, splitter_algorithm):
    print("splitter algorithm:"  + splitter_algorithm)
    best_gain = 0
    best_attribute
    for v in Attributes:
        gain_v = 0
        if splitter_algorithm == "EN":
            gain_v = entropy()
        elif splitter_algorithm == "ME":
            gain_v = majority_error()
        elif splitter_algorithm == "GI":
            gain_v = gini()
        else:
            assert False, "Unknown splitter_algorithm:" + splitter_algorithm + "!!!"
        if gain_v > best_gain:
            best_gain = gain_v
            best_attribute = v
    return best_attribute

# def possible_values():
#     values = {"buying":["vhigh", "high", "med", "low"],"maint": [ "vhigh", "high", "med", "low"], "doors":["2", "3", "4", "5more"], "persons":["2", "4", "more"], "lug_boot":["small", "med", "big"], "safety":["low", "med", "high"]}

atrib_value = {"0":"buying", "1":"maint","2":"doors","3":"persons","4":"lug_boot","5":"safety","6":"label"}
         


# ID3 implementation:

In [115]:
# Input:
# S: the set of Examples
# Label_col_index: column index of the target attribute (the prediction)
# Attributes: the set of measured attributes
def ID3(S, Attributes, Label_col_index, max_tree_level, splitter_algorithm):
    print(Attributes)
    if S[Label_col_index].nunique() == 1:                                      # if all examples have same label: 
        print("Leaf Node: " + S.loc[0,Label_col_index])                        #    Return a leaf node with the label 
    elif len(Attributes) == 0:                                                 # if Attributes empty
        print("Leaf Node: " + S[Label_col_index].mode()[0])                    #    Return a leaf node with the most common label 
    else:
                                                                               # 1. Create a Root node for tree
        A = Best_spliter_attribute(S, Attributes, splitter_algorithm)          # 2. A = attribute in Attributes that best splits S
        for v in S[A].unique():
            S_v = S.loc[S[A] == v]
            if len(S_v) == 0:
                print("Leaf Node: " + S[Label_col_index].mode()[0])
            else:
                print("recursive call")
                Attributes.remove(A)
                ID3(S_v, Attributes,Label_col_index, max_tree_level,splitter_algorithm)

        print("Root Node: " + atrib_value[A])

# main:

In [116]:
Attributes = [0,1,2,3,4,5] # initially put all attributes except the label in Attributes set
ID3(test_df, Attributes,6, 10, "ME")

[0, 1, 2, 3, 4, 5]
splitter algorithm:ME
recursive call
[1, 2, 3, 4, 5]
splitter algorithm:ME
recursive call


ValueError: list.remove(x): x not in list